In [1]:
import random
import gym
import math
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

class DQNCartPoleSolver():
    def __init__(self, n_episodes=1000, n_win_ticks=150, max_env_steps=None, gamma=1.0, epsilon=1.0, epsilon_min=0.01, epsilon_log_decay=0.995, alpha=0.01, alpha_decay=0.01, batch_size=64, monitor=False, quiet=False):
        self.memory = deque(maxlen=100000)
        self.env = gym.make('CartPole-v0')
        if monitor: self.env = gym.wrappers.Monitor(self.env, '../data/cartpole-1', force=True)
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_log_decay
        self.alpha = alpha
        self.alpha_decay = alpha_decay
        self.n_episodes = n_episodes
        self.n_win_ticks = n_win_ticks
        self.batch_size = batch_size
        self.quiet = quiet
        if max_env_steps is not None: self.env._max_episode_steps = max_env_steps

        # Init model
        self.model = Sequential()
        self.model.add(Dense(24, input_dim=4, activation='tanh'))
        self.model.add(Dense(48, activation='tanh'))
        self.model.add(Dense(2, activation='linear'))
        self.model.compile(loss='mse', optimizer=Adam(lr=self.alpha, decay=self.alpha_decay))

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def choose_action(self, state, epsilon):
        return self.env.action_space.sample() if (np.random.random() <= epsilon) else np.argmax(self.model.predict(state))

    def get_epsilon(self, t):
        return max(self.epsilon_min, min(self.epsilon, 1.0 - math.log10((t + 1) * self.epsilon_decay)))

    def preprocess_state(self, state):
        return np.reshape(state, [1, 4])

    def replay(self, batch_size):
        x_batch, y_batch = [], []
        minibatch = random.sample(
            self.memory, min(len(self.memory), batch_size))
        for state, action, reward, next_state, done in minibatch:
            y_target = self.model.predict(state)
            y_target[0][action] = reward if done else reward + self.gamma * np.max(self.model.predict(next_state)[0])
            x_batch.append(state[0])
            y_batch.append(y_target[0])
        
        self.model.fit(np.array(x_batch), np.array(y_batch), batch_size=len(x_batch), verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def run(self):
        scores = deque(maxlen=100)

        for e in range(self.n_episodes):
            state = self.preprocess_state(self.env.reset())
            done = False
            i = 0
            while not done:
                action = self.choose_action(state, self.get_epsilon(e))
                next_state, reward, done, _ = self.env.step(action)
                next_state = self.preprocess_state(next_state)
                self.remember(state, action, reward, next_state, done)
                state = next_state
                i += 1

            scores.append(i)
            mean_score = np.mean(scores)
            if mean_score >= self.n_win_ticks and e >= 100:
                if not self.quiet: print('Ran {} episodes. Solved after {} trials ✔'.format(e, e - 100))
                return e - 100
            if e % 100 == 0 and not self.quiet:
                print('[Episode {}] - Mean survival time over last 100 episodes was {} ticks.'.format(e, mean_score))

            self.replay(self.batch_size)
        
        if not self.quiet: print('Did not solve after {} episodes 😞'.format(e))
        return e

if __name__ == '__main__':
    agent = DQNCartPoleSolver()
    agent.run()


Using TensorFlow backend.
[2017-11-02 15:19:02,807] Making new env: CartPole-v0


[Episode 0] - Mean survival time over last 100 episodes was 14.0 ticks.
[Episode 100] - Mean survival time over last 100 episodes was 106.16 ticks.
[Episode 200] - Mean survival time over last 100 episodes was 126.08 ticks.
[Episode 300] - Mean survival time over last 100 episodes was 139.35 ticks.
Ran 361 episodes. Solved after 261 trials ✔


In [ ]:
state = agent.preprocess_state(agent.env.reset())

done = False
i = 0
e = 0
while not done:
    action = agent.choose_action(state, agent.get_epsilon(e))
    next_state, reward, done, _ = agent.env.step(action)
    next_state = agent.preprocess_state(next_state)
    agent.remember(state, action, reward, next_state, done)
    state = next_state
    i += 1
    e += 1
    agent.env.render(close=False)
print(e)
print(i)
print("yoo")

agent.env.render(close=True)

In [ ]:
env = gym.make('CartPole-v0')
env.reset()
for _ in range(1000):
    env.render()
    env.step(env.action_space.sample()) # take a random action

# env.render(close=True)

In [5]:
agent.predict()

deque([(array([[-0.02988679,  0.02024459,  0.01431086, -0.028473  ]]),
        0,
        1.0,
        array([[-0.02948189, -0.17507963,  0.0137414 ,  0.26869058]]),
        False),
       (array([[-0.02948189, -0.17507963,  0.0137414 ,  0.26869058]]),
        1,
        1.0,
        array([[-0.03298349,  0.01984355,  0.01911521, -0.01962674]]),
        False),
       (array([[-0.03298349,  0.01984355,  0.01911521, -0.01962674]]),
        1,
        1.0,
        array([[-0.03258662,  0.21468623,  0.01872268, -0.30621783]]),
        False),
       (array([[-0.03258662,  0.21468623,  0.01872268, -0.30621783]]),
        0,
        1.0,
        array([[-0.02829289,  0.01930255,  0.01259832, -0.00768953]]),
        False),
       (array([[-0.02829289,  0.01930255,  0.01259832, -0.00768953]]),
        1,
        1.0,
        array([[-0.02790684,  0.21424158,  0.01244453, -0.29637106]]),
        False),
       (array([[-0.02790684,  0.21424158,  0.01244453, -0.29637106]]),
        1,
        